In [24]:
from Loader import fileloader as fl
from Loader import datavalidator as dv
from Loader import datahandledatabase as dhdb
import posixpath

month = '201901'
inifile = fl.ReadIniFile()
defaultpath = inifile.getDefaultPath()
testpath = inifile.getTestPath()

DEBUG:    Values loaded from globals.ini:
DEBUG:    Values loaded from globals.ini:
DEBUG:     * datapath D:/Datos de Usuario/jsaenza/Documents/OneDrive - Entel Peru S.A/MercadoEmpresas
DEBUG:     * datapath D:/Datos de Usuario/jsaenza/Documents/OneDrive - Entel Peru S.A/MercadoEmpresas
DEBUG:     * scriptspath D:/Datos de Usuario/jsaenza/Documents/Comisiones/Comisiones-python
DEBUG:     * scriptspath D:/Datos de Usuario/jsaenza/Documents/Comisiones/Comisiones-python
DEBUG:     * databasepath D:/Datos de Usuario/jsaenza/Documents/Comisiones/SQLiteDB
DEBUG:     * databasepath D:/Datos de Usuario/jsaenza/Documents/Comisiones/SQLiteDB
DEBUG:    defaultpath value is D:/Datos de Usuario/jsaenza/Documents/OneDrive - Entel Peru S.A/MercadoEmpresas/Data Fuente Comisiones/xlsx
DEBUG:    defaultpath value is D:/Datos de Usuario/jsaenza/Documents/OneDrive - Entel Peru S.A/MercadoEmpresas/Data Fuente Comisiones/xlsx
DEBUG:    testpath value is D:/Datos de Usuario/jsaenza/Documents/OneDrive - Entel

In [25]:
parser = inifile.getIniFileParser()

loader = fl.LoadFileProcess(month)
loader.setParser(parser)
loader.setDefaultPath(defaultpath)

In [26]:
# Importando la información
# Validar en los equipos quincenales la jerarquia de ventas
# Copiar los reportes comisionales del periodo : Grandes Cuentas y Pymes

# Inar y eliminando las migraciones


#inarbruto = loader.loadFile('Inar_bruto')
jerarquia = loader.loadFile('Jerarquia')
comisionantes = loader.loadFile('Comisionantes_voz')

dbparser = inifile.getDbIniFileParser()

dbmanager = dhdb.DbDataProcess(month)
dbmanager.setParser(dbparser)
tblempleados = dbmanager.loadData('tblEmpleados')

Archivo: D:/Datos de Usuario/jsaenza/Documents/OneDrive - Entel Peru S.A/MercadoEmpresas/Data Fuente Comisiones/xlsx\201901_Soporte Estructura Inicios Enero.xlsx
Hoja Importada: JERARQUIA
El tamaño de Jerarquia es 11455 registros
El tamaño de Comisionantes_voz es 312 registros
Also setting database database path to D:/Datos de Usuario/jsaenza/Documents/Comisiones/SQLiteDB/mercado_empresas_db.sqlite
Los registros de la tabla tblEmpleados es 1060 registros  


In [27]:
# Armando las validaciones

#valinarfl = {'inarbruto' : inarbruto, 'jerarquia' : jerarquia, 'comisionantes' : comisionantes, 'tblempleados' : tblempleados}
#valinarobj = dv.ValidateInar(valinarfl)
#inarvalidation = valinarobj.validation()

valmultipledatafl = {'jerarquia' : jerarquia, 'comisionantes' : comisionantes, 'tblempleados' : tblempleados}
valmultipledataobj = dv.ValidateMultipleData(valmultipledatafl)
multipledatavalidation = valmultipledataobj.validation()

#Exportando validaciones

#xlsxfile1 = posixpath.join(testpath, month + '_Validaciones en INAR.xlsx')
#exportparams = {'xlsxfile' : xlsxfile1, 'dataframe' : inarvalidation}
#valinarobj.exportValidation(exportparams)

#"""
xlsxfile2 = posixpath.join(testpath, month + '_Otras Validaciones.xlsx')
exportparams = {'xlsxfile' : xlsxfile2, 'dataframe' : multipledatavalidation}
valmultipledataobj.exportValidation(exportparams)
#"""

Archivo exportado D:/Datos de Usuario/jsaenza/Documents/OneDrive - Entel Peru S.A/MercadoEmpresas/Data Fuente Comisiones/test/201901_Otras Validaciones.xlsx con 49 registros


In [ ]:
# Detectando los registros de Tienda que no tienen Orden
str_tienda = 'Tienda'
lst_estados = ['NEWS','ADDS','FCHURN','DEOF','REAC']

mask0 = inarbruto['ESTADO'].isin(lst_estados)
mask1 = inarbruto['NRO_ORDEN'].isnull()
inar_f1 = inarbruto[mask0 & mask1]
mask2 = inar_f1['VENDEDOR'].notnull()
inar_f2 = inar_f1[mask2]
mask3 = inar_f2['VENDEDOR'].str.contains(str_tienda)
inar_f3 = inar_f2[mask3]

#mask2 = inarbruto['VENDEDOR'].str.contains(str_tienda)
#inar_filtrado = inarbruto[mask2]



In [ ]:
inar_f2.to_csv(posixpath.join(testpath, 'inar_sin_orden.csv'))

In [ ]:
tblempleados[tblempleados['CODIGO_INAR'] == 'RPAZ2']

In [ ]:
# Archivos para Sherman
exportar = False
if exportar:
    df1 = inarbruto[:500000]
    df2 = inarbruto[500001:]
    df1.to_csv(testpath + month + '_sherman1.csv', index = False)
    df2.to_csv(testpath + month + '_sherman2.csv', index = False)

In [ ]:
inarbruto[inarbruto['CONTRATO'] == 43035266]
#inarbruto[inarbruto['codigo'] == '1.10108190']
#inarvalidation[inarvalidation['contrato'] == 37838662]

In [ ]:
tblempleados.dtypes